In [65]:
import os
import re
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

from matplotlib import pyplot as plt

import config
import utils

from data import prepare_data
from embeddings import embedder
from llm import smollm_wrapper
from llm import smollm_raw_wrapper
from chains import rag_chain

In [61]:
import importlib
importlib.reload(config)
importlib.reload(prepare_data)
importlib.reload(embedder)
importlib.reload(smollm_wrapper)
importlib.reload(rag_chain)
importlib.reload(utils)

import warnings
warnings.filterwarnings('ignore')

In [3]:
## Initialization of sub-components

## Dataset Loader
dataloader = prepare_data.DataLoader(config.DATASET_NAME)
doc = dataloader.load_documents(split='train')

dataloader_test = prepare_data.DataLoader(config.DATASET_NAME)
doc_test = dataloader_test.load_documents(split='test')

## Embedder Loader
embedder_name = config.EMBED_MODEL
embedder = embedder.Embedder(embedder_name)
vector_store = embedder.build_vectorstore(doc)

## LLM Loader
base_model_id = config.BASE_MODEL_ID
our_pretrained_lm_path = config.MODEL_ID
llm_wrapper = smollm_wrapper.SmolLLMWrapper(base_model_id, our_pretrained_lm_path)

## RAG Chain Loader
rag_chain = rag_chain.RAGChainBuilder(llm_wrapper.llm, vector_store.as_retriever(search_kwargs={"k": 3}))
qa_chain = rag_chain.build_chain()

Initializing Dataset with train set
--------------------
Dataset is preprocessing
--------------------
Dataset preprocessing is done
--------------------
Initializing Dataset with test set
--------------------
Dataset is preprocessing
--------------------
Dataset preprocessing is done
--------------------
Initializing Embedding Model
--------------------
Initializing LLM Model
--------------------
Initializing Retrieval Chain
--------------------


In [4]:
i = 0
print(doc[i])
print(doc_test[i])

page_content='Question: Is there any restriction on withdrawal in rupees of funds held in an EEFC account
Answer: No, there is no restriction on withdrawal in rupees of funds held in an EEFC account. However, the amount withdrawn in rupees shall not be eligible for conversion into foreign currency and for re-credit to the account.'
page_content='Question: Why should I buy Critical Illness insurance
Answer: Critical Illness insurance provides you and your family additional financial security on the diagnosis of a critical illness. The policy provides a lump sum amount which could be used for: Cost of care and treatment Recuperation aid Debts pay off Any lost in income due to a decrease in ability to work View more'


In [5]:
## Make some tests with RAG Chain
query = "What are the criterias to get some loan?"
response = qa_chain.run(query)
print(response)

Question: What are the criterias to get some loan?

Context:
Question: Whom do I contact in case of ay further queries regarding the loan
Answer: You can apply online on the clicking on the below given link, also you can walk into any of our branches located in 11 mentioned locations and our Sales Managers will help you with your need

Question: How will HDFC decide my home loan eligibility
Answer: HDFC assess the customer's repayment capacity based on income, age, qualifications, number of dependants, spouse's income, assets, liabilities, stability and continuity of occupation, and savings history.

Question: Who is eligible for a tractor loan
Answer: Whether you are a farmer or not, you can avail HDFC Bank's Tractor loan for agricultural or commercial purposes. If you are a farmer, you must have a minimum of 3 acres of agricultural land. To know about the eligibility criteria in detail, click here.

Answer: We offer various types of loans under different categories like Personal Loan

### Only Instruction-Tuned SmolLM2

In [6]:
## LLM Loader
base_model_id = config.BASE_MODEL_ID
our_pretrained_lm_path = config.MODEL_ID
llm_raw_wrapper = smollm_raw_wrapper.SmolLLM_Raw_Wrapper(base_model_id, our_pretrained_lm_path)

Initializing LLM Model
--------------------


In [7]:
response_raw = llm_raw_wrapper.generate_response(query)
print(response_raw)

### Instruction:
        You are an AI banking assistant. Respond to the customer's request in a clear and professional manner.

        ### Customer Request:
        What are the criterias to get some loan?

        ### Response:
         I can provide you with information about our loan criteria so that we can assist you better. Here is what your application will need from us:

1. Loan amount (minimum): We have different loan options available, such as personal loans or home equity loans. Please specify how much money you're looking for and which type of loan suits your needs best.
2. Duration: How long do you plan on taking out the loan? This could be short-term (<6 months) or longer term (>3 years). 
3. Repayment terms: Are there any repayment plans or interest rates associated with this loan option? If yes, please let me know if it includes fixed payments or variable repayments based on market conditions.
4. Credit score/credit history: Our lending process considers factors like c

### Evaluation of RAG-based and Only Ins-Tuned SmolLM2 Models on Test Dataset

In [19]:
import evaluate
from datasets import Dataset

In [9]:
# Create the test question-answer pairs
train_questions = [doc_entry.page_content.split("\n")[0].split("Question:")[1] for doc_entry in doc]
test_questions = [doc_entry.page_content.split("\n")[0].split("Question:")[1] for doc_entry in doc_test]
test_answers = [doc_entry.page_content.split("\n")[1].split("Answer:")[1] for doc_entry in doc_test]

print(f"# Train Questions: {len(train_questions)}\n# Test Questions: {len(test_questions)}\n# Test Answers: {len(test_answers)}\n")

# Train Questions: 1411
# Test Questions: 353
# Test Answers: 353



In [10]:
test_questions, test_answers = utils.remove_joint_questions(train_questions, test_questions, test_answers)

print(len(test_questions), len(test_answers))

There are 104 questions in both dataset!
249 249


In [11]:
# Retrieve Contexts in Batch
# Get top_k contexts for all questions

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Get top_k contexts for all questions
retrieved_docs_batch = retriever.batch(test_questions)

print(f"There are {len(retrieved_docs_batch)} documents-retrieved pairs")
print("-"*20)

print(f"A sample question based retrieval:\nQuestion: {test_questions[0]}")
print("-"*20)
print(f"Retrieved documents:\n{retrieved_docs_batch[0]}")
print("-"*20)

There are 249 documents-retrieved pairs
--------------------
A sample question based retrieval:
Question:  What is the maximum deposit amount a Recurring Deposit account can be opened with
--------------------
Retrieved documents:
[Document(page_content='Question: What is the initial deposit amount required to open a Max Current account\nAnswer: You need an initial deposit of Rs. 5,00,000 to open a Max Current account.'), Document(page_content='Question: How to open a Recurring Deposit\nAnswer: Recurring Deposits can now be booked through NetBanking. Following are the steps for booking your Recurring Deposit online: Access your NetBanking account with your Customer ID and IPIN (NetBanking Password) Select Open Recurring Deposit option under the Recurring Deposit menu from the Menu bar located on the left hand side of the web page Select the desired values from the Drop-down lists Once complete, click on Continue and Confirm the details entered'), Document(page_content='Question: Is the

In [ ]:
## Generate Responses with "Only Inst-Tuned SmolLM2"
prompt_template = """
        ### Instruction:
        You are an AI banking assistant. Respond to the customer's request in a clear and professional manner.

        ### Customer Request:
        {question}

        ### Response:
        """

prompts = [
    prompt_template.format(question=q)
    for q in test_questions
]

print(f"There are {len(prompts)} questions in prompt template")
print("-"*20)
print(f"A sample prompt for Only Ins-Tuned SmolL2:\n{prompts[0]}")
print("-"*20)


# Predict responses
inputs = llm_raw_wrapper.base_tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")
outputs = llm_raw_wrapper.tuned_model.generate(**inputs, max_new_tokens=512)

inst_tuned_model_generated_text = llm_raw_wrapper.base_tokenizer.batch_decode(outputs, skip_special_tokens=True)

print(f"There are {len(inst_tuned_model_generated_text)} generated text from Only Ins-Tuned SmolL2")
print("-"*20)
print(f"A sample generated text from Only Ins-Tuned SmolL2:\n{inst_tuned_model_generated_text[0]}")
print("-"*20)


only_inst_tuned_model_generated_answers = []
for onlt_inst_model_generated_text in inst_tuned_model_generated_text:
    only_inst_tuned_model_generated_answers.append(utils.extract_inst_only_model_answer(onlt_inst_model_generated_text))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


There are 249 questions in prompt template
--------------------
A sample prompt for Only Ins-Tuned SmolL2:

        ### Instruction:
        You are an AI banking assistant. Respond to the customer's request in a clear and professional manner.

        ### Customer Request:
         What is the maximum deposit amount a Recurring Deposit account can be opened with

        ### Response:
        
--------------------
There are 249 generated text from Only Ins-Tuned SmolL2
--------------------
A sample generated text from Only Ins-Tuned SmolL2:

        ### Instruction:
        You are an AI banking assistant. Respond to the customer's request in a clear and professional manner.

        ### Customer Request:
         What is the maximum deposit amount a Recurring Deposit account can be opened with

        ### Response:
         I can provide you with the maximum amount that you can deposit with your recurring deposit account. To determine the maximum amount, I'll need some additional in

In [62]:
## Generate Responses with "RAG-based Inst-Tuned SmolLM2"
rag_based_inst_tuned_model_generated_text = []
rag_based_inst_tuned_model_generated_answers = []

test_responses = qa_chain.batch(test_questions[:3])

for test_response_elem in test_responses:
    rag_based_inst_tuned_model_generated_text.append(test_response_elem['result'])

for rag_generated_text in rag_based_inst_tuned_model_generated_text:
    rag_based_inst_tuned_model_generated_answers.append(utils.extract_rag_answer(rag_generated_text))

In [67]:
## Its time to make evaluation

only_inst_tuned_model_generated_answers = rag_based_inst_tuned_model_generated_answers
test_answers = rag_based_inst_tuned_model_generated_answers

df = pd.DataFrame(list(zip(only_inst_tuned_model_generated_answers, rag_based_inst_tuned_model_generated_answers, test_answers)), 
                  columns=['only_inst_model_answer', 'rag_based_model_answer', "gt_answer"])

df.head(2)

,only_inst_model_answer,rag_based_model_answer,gt_answer
0,"Yes, you have that right! The minimum payment ...","Yes, you have that right! The minimum payment ...","Yes, you have that right! The minimum payment ..."
1,It's possible if we know what type of annuity ...,It's possible if we know what type of annuity ...,It's possible if we know what type of annuity ...


In [68]:
## Calculate the ROUGE Score
rouge = evaluate.load('rouge')

only_inst_model_evaluation_result = rouge.compute(
    predictions=only_inst_tuned_model_generated_answers,
    references=test_answers,
    use_aggregator=True,
    use_stemmer=True
)

print(f"Only Instruction Tuned Model Evaluation Result: {only_inst_model_evaluation_result}")

rag_based_model_evaluation_result = rouge.compute(
    predictions=rag_based_inst_tuned_model_generated_answers,
    references=test_answers,
    use_aggregator=True,
    use_stemmer=True
)

print(f"RAG-Based Instruction Tuned Model Evaluation Result: {rag_based_model_evaluation_result}")

Using the latest cached version of the module from C:\Users\citak\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--rouge\b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Sat Aug 31 02:06:36 2024) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


Only Instruction Tuned Model Evaluation Result: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
RAG-Based Instruction Tuned Model Evaluation Result: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
